In [1]:
import telegram
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler, ConversationHandler, MessageHandler, Filters

In [2]:
import pandas as pd
df_bot = pd.read_csv('./telebot.csv')

In [3]:
#This csv shows us the top 10 recommended items for purchase for each existing user.
df_bot.head()

User ID  Item ID
0        0    34888
1        0    40722
2        0   128173
3        0   110908
4        0    30914

In [4]:
token = '862251474:AAFQTrtrpAaqp5fG5vdPdKuEsUu3xMrACps'
updater = Updater(token, use_context=True)

In [5]:
FIRST, SECOND = range(2)

In [6]:
dispatcher = updater.dispatcher
updater.stop()

In [7]:
bot = telegram.Bot(token)
print(bot.get_me())

{'id': 862251474, 'first_name': 'mytaobao!', 'is_bot': True, 'username': 'mytaobao2020_bot'}


In [8]:
#display taobao welcome photo at /start
def start(update, context):
    context.bot.send_photo(chat_id=update.message.chat_id, photo=open('C:/Users/User/Pictures/welcome_taobao.JPG', 'rb'))
    context.bot.send_message(chat_id = update.message.chat_id, text = "Welcome to mytaobao! We provide you recommendations on suitable taobao products for your consideration! How may I address you?")
    return FIRST

In [11]:
def name(txt):
    return "Hi "+ str(txt)+ "! " + "Please provide us your User ID."

In [12]:
name('Shauna')

'Hi Shauna! Please provide us your User ID.'

In [13]:
def response(num):
    try:
        if num.isdigit() == False:
            #num = get_input()
            return 'Invalid input. Please key in valid User ID.'
        if int(num) in range(0,4831):
            return'Welcome back, user ' + str(num) + '. ' + 'We wish to recommend the following items to you: ' + str(list(df_bot[df_bot['User ID']==int(num)]['Item ID']))    
        elif int(num)>=4831: 
            return 'Thank you for your first visit! We wish to recommend the following items to you: [141515, 3423, 137441, 142407, 115747, 208890, 133969, 47310, 134392, 120665]'
        else:
            return 'Please provide a valid User ID, which is a user ID between 0 and 4830.'
    except ValueError:
            return 'Invalid input. Please key in valid User ID.'

In [14]:
#Test: existing user
response('20')

'Welcome back, user 20. We wish to recommend the following items to you: [44512, 174403, 28918, 69322, 168254, 171760, 223186, 15477, 136029, 112725]'

In [15]:
#Test: new user
response('8000')

'Thank you for your first visit! We wish to recommend the following items to you: [141515, 3423, 137441, 142407, 115747, 208890, 133969, 47310, 134392, 120665]'

In [16]:
#Invalid user
response('hello')

'Invalid input. Please key in valid User ID.'

In [17]:
#Invalid user
response('5.5')

'Invalid input. Please key in valid User ID.'

In [20]:
def nameMsg(update, context):
    user = update.message.from_user
    context.bot.send_message(chat_id=update.effective_chat.id, 
                              text=name(update.message.text))
    return SECOND



In [21]:
def userMsg(update, context):    
    user = update.message.from_user
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             # text=update.message.text)
                              text=response(update.message.text))   
    return



In [22]:
#display help message at /help
def help(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text = "Hi, if you are an existing user, please key in your user ID between 0 to 4830. We would then recommend you the top 10 items to purchase based on your user preference history. If you are a new user, we wish to introduce you the top 10 most hot selling products on our website.")

In [23]:
help_command_handler = CommandHandler('help', help)
dispatcher.add_handler(help_command_handler)

In [24]:
#display taobao goodbye photo at /exit
def exit(update, context):
    context.bot.send_photo(chat_id=update.message.chat_id, photo=open('C:/Users/User/Pictures/byetaobao.JPG', 'rb'))


In [25]:
end_command_handler = CommandHandler('exit', exit)
dispatcher.add_handler(end_command_handler)


In [26]:
conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        FIRST: [MessageHandler(Filters.text, nameMsg, pass_user_data=True), CommandHandler('help', help)],
        SECOND: [MessageHandler(Filters.text, userMsg, pass_user_data=True), CommandHandler('help', help)]
    },
    fallbacks=[CommandHandler('exit', exit)]
)

In [27]:
dispatcher.add_handler(conv_handler)

In [28]:
updater.start_polling()